# Compositional Pattern Producing Networks (CPPNs)

CPPNs are a type of artificial neural network that generate patterns by taking spatial coordinates as inputs and producing outputs that can be interpreted as pixel intensities, colors, or other attributes relevant to visual patterns. CPPNs are characterized by their ability to produce complex, high-resolution, and aesthetically interesting designs that exhibit symmetries and other natural characteristics, often used in the fields of evolutionary art and design. They differ from traditional neural networks in that their architecture can include a variety of activation functions, which are chosen to enhance the patterns' complexity and diversity.

<img 
    style="display: block; 
           margin-left: auto;
           margin-right: auto;
           width: 60%;"
    src="../Examples/CPPN_Visuals.png" 
    alt="Our logo">
</img>

CPPNS are typically evolved using the NeuroEvolution of Augmenting Topologies (NEAT) algorithm, which is an evolutionary algorithm that creates and optimizes neural networks. Developed by Kenneth O. Stanley, NEAT starts with a simple initial population of neural networks and evolves them over time through genetic algorithms. It innovatively combines both the topology and the weights of the networks, allowing for the evolution of increasingly complex structures from simple beginnings. NEAT's key features include protecting innovation through speciation, incrementally growing from minimal structure, and using historical markings to track genes across generations, making it highly effective for tasks where both the structure and weights of a neural network need to be optimized simultaneously.

CPPN-NEAT implementation from [neat-python](https://github.com/CodeReclaimers/neat-python).

## Initializing NEAT-Python

First, we must import the neccessary libraries, and load the configuration file used by NEAT python to evolve genomes.

In [ ]:
import neat
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax

def load_config(filename):
    genome = neat.DefaultGenome
    reproduction = neat.DefaultReproduction
    species_set = neat.DefaultSpeciesSet
    stagnation_method = neat.DefaultStagnation
    try:
        config = neat.Config(genome, reproduction, species_set, stagnation_method, filename)
    except:
        print("Config file could not be found, please make sure the filename is correct and exists in the target directory.")
        config = None
    return config

config = load_config(f"../Examples/neat.cfg")

## Generating the Phenotype

In this notebook, our phenotype is a 3D lattice of voxels (i.e., 3D pixels) which can be in one of two states, on or off.  To generate a lattice, we iterate over the XYZ coordinates and feed these values to the genome (CPPN), recording its output. We apply a sigmoid activation function to the final output to ensure the value is between 0 and 1, and round it to the nearest integer to indicate the presence of a voxel.

In [ ]:
def create_cppn_neat(genome, config, size=20):
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    voxels = np.zeros((size, size, size), dtype=float)
    for x in range(size):
        for y in range(size):
            for z in range(size):
                input_coordinates = (x / size * 2 - 1, y / size * 2 - 1, z / size * 2 - 1)
                output = net.activate(input_coordinates)[0]
                sigmoid_output = np.round(1 / (1 + np.exp(-output)))
                voxels[x, y, z] = sigmoid_output > 0.5
    return voxels


## Defining our fitness function, and the evaluation loop

In this example, we evaluate genomes based on the diversity of their output. For diversity, we use the Kullback-Leibler (KL) divergence metric. We assign fitness by calculating the KL divergence to every other individual in the population, and take the average of the K (10) nearest neighbors.

In [ ]:
k = 10
lattice_size = 5

def kl_divergence(p, q):
    return np.sum(p * np.log(p / q))

def eval_genomes(genomes, config):

    voxel_distributions = {}

    # Generate the phenotypes in advance to save on redundant computation
    for genome_id, genome in genomes:
        voxel_distribution = softmax(create_cppn_neat(genome, config, lattice_size).ravel())
        voxel_distributions.update({genome_id: voxel_distribution})

    for genome_id, genome in genomes:
        voxel_distribution = voxel_distributions[genome_id]
        distances = []

        if np.sum(voxel_distribution) == 0:
            genome.fitness = -1000
        else:
            for other_id, _ in genomes:
                target_distribution = voxel_distributions[other_id]
                if np.sum(target_distribution) == 0:
                    continue
                if genome_id == other_id:
                    continue
                distances.append(kl_divergence(voxel_distribution, target_distribution))
            distances = sorted(distances, reverse=True)
            genome.fitness = np.mean(distances[:k])

## Running NEAT

Next, we run NEAT for the specified number of generations, and attach a reporter to output results after every generation, and a statistics tracker provided by the NEAT-Python library.

In [ ]:
p = neat.Population(config)
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
winner_genome = p.run(eval_genomes, 50)


## Genome Complexity

For the sake of this example notebook, we set the probability to add nodes/connections to 25%, and the probability to remove nodes to 0%. This means the networks should mostly grow in both the number of nodes and number of connections over time. We plot these results by looking at the most fit genomes from each generation and visualizing these properties on a graph.

In [ ]:
def plot_complexity(statistics):
    generation = range(len(statistics.most_fit_genomes))
    num_nodes = [c.size()[0] for c in statistics.most_fit_genomes]
    num_conns = [c.size()[1] for c in statistics.most_fit_genomes]

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(generation, num_nodes)
    plt.title('Node Complexity Over Generations')
    plt.xlabel('Generations')
    plt.ylabel('Number of Nodes')

    plt.subplot(1, 2, 2)
    plt.plot(generation, num_conns)
    plt.title('Connection Complexity Over Generations')
    plt.xlabel('Generations')
    plt.ylabel('Number of Connections')
    plt.show()

plot_complexity(stats)

## Visualizing Phenotype

Finally, we can also visualize the outputs of the genome using matplotlib. In the below example, we take the best genome from the final generation and plot it in 3D using the voxel() function. 

In [ ]:
def voxel_plot(lattice, title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.voxels(lattice, edgecolor="k")
    ax.set_title(title)
    ax.set_axis_off()
    plt.show()

winner_voxels = create_cppn_neat(winner_genome, config)
voxel_plot(winner_voxels, 'Most Unique Lattice')

## Best Practices

### 1. Appropriate Fitness Function
Choose or design a fitness function that meaningfully relates to the task at hand. For CPPNs, which often generate patterns or structures, the fitness function should appropriately evaluate the quality or usefulness of these outputs. Ensure the fitness function is scaled or normalized to handle the range of possible output values, especially when comparing across different scales of complexity.

### 2. Start Simple
Incremental Growth: Begin with simple network architectures and allow NEAT to incrementally increase complexity. This avoids premature convergence on suboptimal architectures and promotes thorough exploration of simpler solutions that might be more efficient.\

### 3. Diversity Through Speciation
Use speciation to protect new mutations, allowing them to mature. This is crucial for maintaining genetic diversity within the population, which is a core advantage of NEAT. Tune speciation parameters (compatitibility threshold in the configuration file) carefully to balance between too much and too little competition within species.

### 4. Topology and Mutation Rates
Experiment with different mutation rates for adding nodes and connections. The balance between exploration (trying new things) and exploitation (refining existing solutions) is key. Consider implementing dynamic adjustment of mutation rates based on the performance progress over generations.

### 5. Parameter Tuning
NEAT has several parameters (e.g., crossover rates, mutation rates, compatibility threshold for speciation). Systematically tune these parameters for your specific application. Employ grid search or more advanced techniques like Bayesian optimization to find optimal parameter sets.

### 6. Scalability and Efficiency
Utilize parallel computing resources to evaluate genomes in parallel, reducing the total evolutionary time and increasing scalability.

### 7. Visualization and Analysis
Visualize and analyze the evolution process regularly (e.g., changes in genome complexity, fitness landscapes). This can provide insights into evolutionary dynamics and help in adjusting strategies. Regularly inspect the outputs (patterns, structures) generated by CPPNs for insights that could lead to manual tweaks in the network or evolutionary process.
